# Peer-graded Assignment: Analyzing Historical Stock/Revenue Data and Building a Dashboard

**Student**: Mk  
**Date**: 2025-06-13

This notebook extracts historical stock and revenue data for Tesla (TSLA) and GameStop (GME), displays key snapshots, and builds an interactive dashboard to visualize price and revenue trends.

## Install Required Packages

In [7]:
!pip install yfinance requests bs4 plotly


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Question 1: Use yfinance to Extract Tesla Stock Data

In [8]:
import yfinance as yf
import pandas as pd

# Create ticker object and fetch historical data
tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")

# Reset index and display first five rows
tesla_data.reset_index(inplace=True)
print("Tesla Stock Data (first 5 rows):")
print(tesla_data.head())

Tesla Stock Data (first 5 rows):
                       Date      Open      High       Low     Close  \
0 2010-06-29 00:00:00-04:00  1.266667  1.666667  1.169333  1.592667   
1 2010-06-30 00:00:00-04:00  1.719333  2.028000  1.553333  1.588667   
2 2010-07-01 00:00:00-04:00  1.666667  1.728000  1.351333  1.464000   
3 2010-07-02 00:00:00-04:00  1.533333  1.540000  1.247333  1.280000   
4 2010-07-06 00:00:00-04:00  1.333333  1.333333  1.055333  1.074000   

      Volume  Dividends  Stock Splits  
0  281494500        0.0           0.0  
1  257806500        0.0           0.0  
2  123282000        0.0           0.0  
3   77097000        0.0           0.0  
4  103003500        0.0           0.0  


## Question 2: Use Webscraping to Extract Tesla Revenue Data

In [ ]:
# %% 
# Question 2: Use Webscraping to Extract Tesla Revenue Data (with headers + pd.read_html)
import requests
import pandas as pd

url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
headers = {"User-Agent": "Mozilla/5.0"}
resp = requests.get(url, headers=headers)

# Parse all tables from the fetched HTML
tables = pd.read_html(resp.text)

# Find the one with exactly two columns (Date & Revenue)
for df in tables:
    if df.shape[1] == 2:
        tesla_revenue = df.copy()
        break

# Rename, clean and convert
tesla_revenue.columns = ["Date", "Revenue"]
tesla_revenue["Revenue"] = (
    tesla_revenue["Revenue"]
    .str.replace(r"[\$,]", "", regex=True)
    .astype(int)
)
tesla_revenue["Date"] = pd.to_datetime(tesla_revenue["Date"])

# Show last 5 rows
print("Tesla Revenue Data (last 5 rows):")
print(tesla_revenue.tail())

HTTPError: HTTP Error 403: Forbidden

## Question 3: Use yfinance to Extract GameStop Stock Data

In [ ]:
# Create ticker object and fetch historical data for GameStop
gme = yf.Ticker("GME")
gme_data = gme.history(period="max")

# Reset index and display first five rows
gme_data.reset_index(inplace=True)
print("GameStop Stock Data (first 5 rows):")
print(gme_data.head())

## Question 4: Use Webscraping to Extract GameStop Revenue Data

In [ ]:
# Scrape GameStop quarterly revenue from Macrotrends
url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"
html = requests.get(url).text
soup = BeautifulSoup(html, "html.parser")

# Locate and parse the revenue table
tables = soup.find_all("table")
for table in tables:
    if "GameStop Quarterly Revenue" in table.text:
        revenue_table = table
        break

gme_rev_rows = revenue_table.find_all("tr")[1:]
rows = []
for row in gme_rev_rows:
    cols = row.find_all("td")
    if len(cols) >= 2:
        date = cols[0].text.strip()
        revenue = cols[1].text.strip().replace("$", "").replace(",", "")
        if revenue:
            rows.append([date, int(revenue)])
gme_revenue = pd.DataFrame(rows, columns=["Date", "Revenue"])
gme_revenue["Date"] = pd.to_datetime(gme_revenue["Date"])

print("GameStop Revenue Data (last 5 rows):")
print(gme_revenue.tail())

## Question 5 & 6: Plot Stock and Revenue Graphs

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Function to build interactive dashboard
def make_graph(stock_df, revenue_df, title):
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        subplot_titles=(f"{title} Stock Price", f"{title} Revenue"),
        vertical_spacing=0.2
    )
    fig.add_trace(
        go.Scatter(x=stock_df["Date"], y=stock_df["Close"], name="Close Price"),
        row=1, col=1
    )
    fig.update_xaxes(rangeslider_visible=True, row=1, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)

    fig.add_trace(
        go.Bar(x=revenue_df["Date"], y=revenue_df["Revenue"], name="Revenue"),
        row=2, col=1
    )
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)

    fig.update_layout(
        title_text=f"{title} Historical Stock Price and Revenue",
        height=800
    )
    fig.show()

# Plot for Tesla
make_graph(tesla_data, tesla_revenue, "Tesla")
# Plot for GameStop
make_graph(gme_data, gme_revenue, "GameStop")

## Assignment Repository Link

[GitHub Repository](https://github.com/mrigankmaharana/IBM_PythonProjectforDataScience)